In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from scipy.optimize import curve_fit
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern",
    "figure.dpi": 100,
})

from Finite_volume_method_1D import *

In [ ]:
chippath = '../../DataKian/bTa_LT278W2chip5/'
filename = 'KID5_103dBm__TmK20_avgpulse_ampphase.csv'
length = 90

lambdanm = 402
KID402 = KID_data(chippath,lambdanm,filename,length)

lambdanm = 673
KID673 = KID_data(chippath,lambdanm,filename,length)

lambdanm = 986
KID986 = KID_data(chippath,lambdanm,filename,length)

lambdanm = 1545
KID1545 = KID_data(chippath,lambdanm,filename,length)

In [ ]:
KID402.fit_tail(start=150,end=250)
KID673.fit_tail(start=150,end=250)
KID986.fit_tail(start=150,end=250)
KID1545.fit_tail(start=130,end=230)

KIDdict = {
    402 : KID402,
    673 : KID673,
    986 : KID986,
    1545 : KID1545
}

In [ ]:
print(KID402.tauqpstar,KID402.tauqpstarstd)
print(KID673.tauqpstar,KID673.tauqpstarstd)
print(KID986.tauqpstar,KID986.tauqpstarstd)
print(KID1545.tauqpstar,KID1545.tauqpstarstd)
print(KID402.L)
print(KID673.L)
print(KID986.L)
print(KID1545.L)

L=np.mean([KID402.L,KID673.L,KID986.L,KID1545.L])

In [ ]:
simtime = 80
leeway = 0

FITx = np.arange(simtime-leeway)*KID402.dt
FITy = {}
for l in [402,673,986,1545]:
    indmin = np.argmax(KIDdict[l].phase)
    indmax = indmin + simtime - leeway
    FITy[l] = KIDdict[l].phase[indmin:indmax]

plotdatay = np.array([FITy[402],FITy[673],FITy[986],FITy[1545]])
fitdatay = plotdatay.ravel()

def func_to_parallelize(t,K,phi_init,l,KID):
     SIM = KID_sim(KID,0.13,K,phi_init/l,0.01,1/4,sigma_IC=2,L=L,simtime_approx=simtime)
     return np.interp(t,SIM.t_axis,SIM.timeseriestheta)
     
def fitfunc(t,D1,D2,D3,D4,K,IC):
     datalist = Parallel(n_jobs=4)(delayed(func_to_parallelize)(t,K,IC,l,KIDdict[l]) for l in [402,673,986,1545])
     return np.concatenate(datalist)

In [ ]:
plotfit = fitfunc(FITx,25.4,3.7,0.4,0.1,110,3122)
plotfit = np.reshape(plotfit,plotdatay.shape)

In [ ]:
plt.figure()
for y,fit in zip(plotdatay,plotfit):
    plt.scatter(FITx,y,s=1)
    plt.semilogy(FITx,fit)
plt.show()

In [ ]:
plt.figure()
for y,fit in zip(plotdatay,plotfit):
    plt.scatter(FITx,y,s=1)
    plt.plot(FITx,fit)
plt.show()